# Traitement et Pré-traitement de Texte (NLP)

Ce notebook résume les techniques fondamentales de traitement du langage naturel (NLP) : la tokenisation, la gestion des stop words, la normalisation, ainsi que la lemmatisation et le stemming.

## 1. Tokenization

La tokenisation consiste à diviser une chaîne de caractères ou un texte en unités plus petites appelées **tokens**. C'est une étape essentielle pour réduire la taille du texte brut et faciliter l'analyse statistique.

Il existe plusieurs types de tokenisation :

* **Par mots** : Divise le texte en mots individuels.

* **Par phrases** : Divise un paragraphe en phrases distinctes.

* **Par caractères** ou **N-gram**.

In [1]:
from nltk.tokenize import word_tokenize, sent_tokenize

# --- 1. Tokenization par mots ---
text_word = "I am back on the track."
token_words = word_tokenize(text_word)
print(f"Résultat de la tokenization par mots : {token_words}")

Résultat de la tokenization par mots : ['I', 'am', 'back', 'on', 'the', 'track', '.']


In [2]:
# --- 2. Tokenization par phrases ---
text_sent = "I am you. You are not me."
token_sentences = sent_tokenize(text_sent)
print(f"Résultat de la tokenization par phrases : {token_sentences}")

Résultat de la tokenization par phrases : ['I am you.', 'You are not me.']


*Source du code : `tokenization.py`*

## 2. Stop Words (Mots Vides)

Les "Stop Words" sont des mots courants (articles, prépositions, pronoms) qui apportent peu de valeur sémantique (ex: "le", "et", "dans").
Leur suppression permet d'optimiser les ressources informatiques et de se concentrer sur les mots-clés importants, bien que cela dépende de la tâche (à éviter pour la traduction automatique par exemple).

Nous allons voir trois implémentations différentes : NLTK, Spacy et Gensim.

In [ ]:
# --- Méthode 1 : NLTK ---
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

text = "Your key does not open the door"
stop_words_set = set(stopwords.words('english'))

tokens = word_tokenize(text.lower())
filtered_tokens_nltk = [word for word in tokens if word not in stop_words_set]

print("Original:", tokens)
print("Filtré avec NLTK:", filtered_tokens_nltk)

Original: ['your', 'key', 'does', 'not', 'open', 'the', 'door']
Filtré avec NLTK: ['key', 'open', 'door']


In [ ]:
# --- Méthode 2 : Spacy ---
import spacy
try:
    nlp = spacy.load("en_core_web_sm")
    doc = nlp(text)
    filtered_words_spacy = [token.text for token in doc if not token.is_stop]
    print("Filtré avec Spacy:", filtered_words_spacy)
except OSError:
    print("Le modèle Spacy 'en_core_web_sm' n'est pas installé.")

Filtré avec Spacy: ['key', 'open', 'door']


In [5]:
# --- Méthode 3 : Gensim ---
from gensim.parsing.preprocessing import remove_stopwords

new_filtered_text_gensim = remove_stopwords(text)
print("Filtré avec Gensim:", new_filtered_text_gensim)

Filtré avec Gensim: Your key open door


*Source du code : `stop_words.py`*

## 3. Lemmatization et Stemming

Ces deux techniques servent à réduire les mots à leur forme racine, mais elles fonctionnent différemment.

### Stemming (Racinisation)
C'est une méthode basée sur des règles (troncature). Elle coupe les suffixes (ex: "-ing", "-ed") de manière mécanique. Le résultat n'est pas toujours un mot valide (ex: "studi" pour "studies").

* *Avantage* : Rapide, utile pour les moteurs de recherche.

* *Désavantage* : Moins précis sémantiquement.

### Lemmatization
C'est une méthode basée sur la linguistique. Elle utilise un dictionnaire et l'analyse grammaticale pour retrouver le **lemme** (la forme du dictionnaire).

* *Avantage* : Préserve le sens (ex: "better" devient "good").

* *Désavantage* : Plus coûteux en calcul.

In [6]:
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

# --- Stemming ---
stemmer = PorterStemmer()
words_to_stem = ["learning", "comptutationally", "engineering", "studies"]
stemmed_words = [stemmer.stem(word) for word in words_to_stem]

print(f"Résultats après Stemming : {stemmed_words}")

Résultats après Stemming : ['learn', 'comptut', 'engin', 'studi']


In [ ]:
# --- Lemmatization ---
lemmatizer = WordNetLemmatizer()

lemmas = [
    lemmatizer.lemmatize("running", pos=wordnet.VERB), # v -> run
    lemmatizer.lemmatize("better", pos=wordnet.ADJ),   # a -> good
    lemmatizer.lemmatize("studies", pos=wordnet.NOUN), # n -> study
    lemmatizer.lemmatize("was", pos=wordnet.VERB)      # v -> be
]
print(f"Résultat Lemmatization : {lemmas}")

Résultat Lemmatization : ['run', 'good', 'study', 'be']


*Source du code : `lem_stem.py`*

## 4. Normalisation Complète (Pipeline)

La normalisation transforme le texte en une forme canonique standardisée. Il n'y a pas de procédure universelle, mais les étapes classiques incluent :

1.  Mise en minuscules (Lowercasing).

2.  Suppression des chiffres et de la ponctuation (via Regex).

3.  Suppression des espaces superflus.

4.  Suppression des Stop Words.

Voici un exemple complet sur un texte "sale" contenant des logs, des dates et du bruit.

In [ ]:
import re
from nltk.corpus import stopwords

input_text = "      IN 2025, following the massive DATA BREACH disclosed on 07/18/2025, investigators revealed that over 1.500.000 records—including user_IDs (e.g., U-99821#A), hashed passwords (SHA256$9f2@!), phone numbers, and transactions totaling €12,008—were exfiltrated via an unsecured API endpoint (/v1/export?debug=true), triggering alerts at 02:14:33 UTC, regulatory fines under GDPR_ART.33, internal emails marked “URGENT!!!”, incident_refs=[INC-2025-LEAK-Ω], and a public statement asserting “NO SYSTEM IS 100% IMMUNE,” punctuated by logs showing latency=1.42s, retries=5/10, and access from IPs like 185.203.44.7 before containment  ."

stop_words = set(stopwords.words('english'))

text_processed = input_text.lower()

text_processed = re.sub(r'\d+', '', text_processed)

text_processed = re.sub(r'[^\w\s]', '', text_processed)

text_processed = text_processed.strip()

lst_input_text = text_processed.split()
print(f"Liste intermédiaire : {lst_input_text[:10]}...")

final_tokens = [word for word in lst_input_text if word not in stop_words]
final_text = " ".join(final_tokens)

print("\n--- Texte Final Normalisé ---")
print(final_text)

Liste intermédiaire : ['in', 'following', 'the', 'massive', 'data', 'breach', 'disclosed', 'on', 'investigators', 'revealed']...

--- Texte Final Normalisé ---
following massive data breach disclosed investigators revealed recordsincluding user_ids eg ua hashed passwords shaf phone numbers transactions totaling exfiltrated via unsecured api endpoint vexportdebugtrue triggering alerts utc regulatory fines gdpr_art internal emails marked urgent incident_refsincleakω public statement asserting system immune punctuated logs showing latencys retries access ips like containment


*Source du code : `text_normalization.py`*